# 통계적 가설검정

모집단 모수에 대한 가설을 세우고 표본으로부터 계산되는 통계량을 이용하여 가설이 옳은지 판단하는 통계적 방법이다. 

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

%precision 3
np.random.seed(1111)

## 모평균 가설검정

만약 예를들어 한 패스트푸드점에서 팔고 있는 감자튀김이 130g보다 작은지 검정하는 예제를 살펴보자.  
여기서 감자튀김은 정규분포를 따르고 모분산이 9임을 알고있다고 가정해 보자. 표본의 갯수 n = 9 

In [5]:
df = pd.read_csv('./data/ch11_potato.csv')
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [6]:
# 표본의 평균 = 모평균의 점추정량
s_mean = np.mean(sample)
s_mean

128.4507142857143

## 통계적 가설검정이란

모수에 대한 귀무가설과 대립가설을 세우고 표본으로부터 계산되는 통계량을 이용하여 어느 가설이 옳은지 판단하는 통계적 방법이다.

### 통계적 가설검정의 흐름

감자튀김에 관해 확인하고자하는 바는 모평균이 130g보다 적은지 여부이다. 
* 귀무가설 : 모평균이 130g이다.
* 대립가설 : 모평균은 130g보다 작다.

일단 감자튀김의 표본평균은 $ \bar{X} \thicksim N\left(130, \dfrac{9}{14}\right)$ 분포를 따른다.

$ \bar{X}$ 가 $P(\bar{X} \le x) = 0.05$를 만족하는 x를 생각해보자.

In [7]:
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

$ \bar{X}$ 가 $P(\bar{X} \le 128.681) = 0.05$가 되고 표본평균이 128.681g 이하의 무게가 되는것은 5%의 확률로 발생한다. 따라서 위 표본의 평균 무게가 128.45g으로 이것은 5% 확률로 발생하는 드문 사건이라고 할 수 있다. 

#### 모평균 가설검정 - 검정 통계량의 계산 

모분산을 알고 있는 경우이므로 검정통계량은 다음과 같이 계산된다.
$$ Z = \dfrac{\bar{X}-\mu_0}{\sigma/\sqrt{n}} $$

In [5]:
z = (s_mean - 130) / np.sqrt(9/14) #검정통계량
z

-1.932298779026813

In [6]:
rv = stats.norm() # 임계값
rv.isf(0.95)

-1.6448536269514722

검정통계량과 임계값을 비교하면 검정통계량이 작은 값이다. 이에 따라 귀무가설은 기각되고 평균은 130g보다 작다는 결론에 이른다.   

p-value는 누적분포함수를 사용해서 구할 수 있다. p-value가 유의수준 $\alpha$=0.05보다 작으므로 귀무가설을 기각한다는 동일한 결론이다. 

In [7]:
rv.cdf(z)

0.026661319523126635

### 단측검정과 양측검정

만약 대립가설 "H1: 모평균은 130g이 아니다."라고 한다면 양측검정을 시행한다.  
유의수준 5%에서 양측검정을 시행할때 표준정규분포의 95%구간에 따라 구할 수 있다. 

In [8]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [9]:
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

이렇게 구해진 임계값과 검정통계량을 비교해보면 검정통계량이 채택역안에 있다는 것을 알수 있다. 따라서 양측검정에서는 귀무가설을 기각할 수 없다. 즉, 감자튀김의 모평균은 130g과 다르지 않다.


양측검정의 p-value를 구할때는 양쪽 면적을 고려할 필요가 있으므로 누적밀도함수의 값을 2배로 한다. 

In [10]:
rv.cdf(z) * 2

0.05332263904625327

p-value 가 유의수즌 0.05보다 크므로 귀무가설을 기각하지 않는다.

### 가설검정에서의 두 가지 오류

* 제1종오류 : 귀무가설이 옳은데 귀무가설을 기각하는 오류
* 제2종오류 : 대립가설이 옳은데 귀무가설을 채택하는 오류

#### 1. 제 1종 오류
제 1종오류가 어느 정도의 비율로 발생하는 지 시물레이션을 한다. 즉, 모평균이 130g인데 "모평균은 130g보다 작다" 는 결론을 내리는 비율을 계산해 본다.

In [11]:
rv = stats.norm(130, 3)

In [12]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.053

제 1종 오류를 범한 비율이 0.053 이다. 제1종 오류를 범할 확률을 "유의수준" 이라고하고 $\alpha$로 표시한다.   
검정시 이 유의수준을 먼저 정하고 검정을 시작해야 한다. 

#### 2. 제 2 종 오류

실제로 "모평균이 130g보다 작다" 인데 "모평균은 130g보다 작다"라는 결론을 얻을 수 없는 비율을 계산하면 다음과 같다.  
만약 실제로 감자튀김의 모평균이 128g 이라고 하고 계산하자. 

In [13]:
rv = stats.norm(128, 3) # 모평균이 128g이다.

In [14]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
        
cnt / n_samples

0.197

제 2종오류의 비율은 0.197이다. 제 2종오류를 범할 확률은 $\beta$라고 한다. 그러나 이 경우 우리는 모집단에 대해서 실제로 모평균이 130g보다 작은지 알 수 없기에 $\beta$는 분석자가 제어할 수 있는 영역이 아니다.  
이와 같이 가설검정에서 제1종 오류는 제어할 수 없지만 제2종 오류는 제어할 수 없다는 비대칭성이 있다. 

## 가설검정

### 1. 정규분포의 모평균에 대한 검정(모분산을 알고 있음)

* H0 : $\mu = \mu_0$
* H1 : $\mu \ne \mu_0$ (양측검정)

검정통계량은 
$$ Z = \dfrac{\bar{X}-\mu_0}{\sigma/\sqrt{n}} $$ 이다. 

이 식을 파이썬으로 그대로 구현하면 다음과 같다.

In [15]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)

    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p값은 {p:.3f}')

위 함수를 이용하여 감자튀김 표본에 대하여 "모평균이 130g"인지 양측검정을 수행하면 결과는 다음과 같다. 

In [16]:
pmean_test(sample, 130, 9)

귀무가설을 채택
p값은 0.053


### 2. 정규분포의 모분산에 대한 검정

모분산의 검정은 모본산이 어떤 값$\sigma^2_0$이 아닌것을 주장(대립가설)하기 위한 검정이다.  

이 검정에서는 다음과 같은 검정통계량을 사용한다. 

$$ \dfrac{(n-1)S^2}{\sigma^2} $$ 
이 검정통계량은 $ \chi^2(n-1)$ 을 따른다.  

이를 파이썬으로 구현하면 다음과 같다.

In [17]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p값은 {p:.3f}')

감자튀김의 표본데이터를 사용하여 모분산 $\sigma^2 = 9$ 인지 검정하면 다음과 같다. 

In [18]:
pvar_test(sample, 9)

귀무가설을 채택
p값은 0.085


### 3. 정규분포의 모평균에 대한 검정(모분산을 알지 못함) - 일표본 t-검정

모분산을 모르므로 기본적으로 t분포를 사용한다. 모표준편차 σ의 추정량 S로 대치한 다음의 검정통계량 식을 사용한다.

$$ t = \dfrac{(\bar{X}-\mu_0)}{\sqrt{s^2/n}} $$

이 식은 자유도가 n-1인 t-분포를 따른다.

이를 파이썬으로 구현하면 다음과 같다.

In [19]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

In [20]:
pmean_test(sample, 130)

귀무가설을 채택
p값은 0.169


#### ttest_1samp 함수

이러한 t 검정은 scipy.stats에 ttest_1samp 함수로 구현되어 있다.  
이 함수의 return 값은 t 검정 통계량과 p-value이다. 

In [21]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.4551960206404198, 0.16933464230414275)

# 실습

1. 다음과 같은 과자 무게 데이터가 있다.

    * 과자 무게의 평균을 구하시오.
    * "과자의 평균 무게는 50g이다"라는 귀무가설을 유의수준 5%에서 검정하시오.(stats.ttest_1samp 함수를 사용하시오)

```
data = pd.Series([58.53 , 52.353, 74.446, 52.983, 55.877, 67.66 , 47.726, 50.267,
       56.501, 52.361, 45.458, 53.361, 52.129, 59.828, 41.682, 49.399,
       64.211, 69.859, 42.911, 60.159])
       
data
```

    